Installing the large spacy model files that contains word2vec

In [1]:
!pip install spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=250cd72a3cb33492b134e211bb5e23ff11a00b2495a0c044e35c2de9b82b26cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqv8g8vq/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
stopwords = nlp.Defaults.stop_words

Example text

In [3]:
text1 = "This is cat"
text2="This is Dog"
doc1=nlp(text1)
doc2=nlp(text2)

Similarity between two sentences

In [4]:
doc1.similarity(doc2)

0.9446337664070549

Similarity between Words

In [5]:
for token1 in doc1:
  for token2 in doc2:
    print(token1.text,token2.text,token1.similarity(token2))

This This 1.0
This is 0.5970357
This Dog 0.31190354
is This 0.5970357
is is 1.0
is Dog 0.33961135
cat This 0.2768825
cat is 0.28220233
cat Dog 0.80168545


Sentiment Analysis using the spam classification file

Importing necessary libraries

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/spam_or_not_spam.csv")

In [8]:
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [9]:
data.shape

(3000, 2)

In [10]:
data.isnull().sum()

email    1
label    0
dtype: int64

In [11]:

data=data.dropna()

Removing punctuations and numbers from the given file

In [12]:
import re
def text_preprocessing(sentence):
  x=str(sentence).lower().replace('//',' ').replace('_',' ').replace('.',' ').replace(',',' ').replace('\"',' ').replace('!',' ').replace('@',' ').replace('#',' ').replace('$',' ').replace('%',' ').replace('^',' ').replace('&',' ').replace('*',' ').replace('(',' ').replace(')',' ')
  x=re.sub(r'\[[0-9]*\]',' ',x)
  return x

In [13]:
data['email']=data['email'].apply(lambda x:text_preprocessing(x))

Get the vector for the word

In [14]:
def get_vector(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [15]:
data['vector']=data['email'].apply(lambda x: get_vector(x))

In [16]:
data.head()

,email,label,vector
0,date wed number aug number number number numb...,0,"[0.008441204, 0.11537735, -0.1572698, -0.01277..."
1,martin a posted tassos papadopoulos the greek ...,0,"[0.068682395, 0.14540102, -0.12883942, -0.0375..."
2,man threatens explosion in moscow thursday aug...,0,"[0.0026262603, 0.10563048, -0.033479653, 0.036..."
3,klez the virus that won t die already the most...,0,"[-0.107610874, 0.06578168, -0.09968269, -0.089..."
4,in adding cream to spaghetti carbonara which ...,0,"[-0.044400677, 0.1261134, -0.123361, -0.017362..."


In [17]:
X=data['vector'].to_numpy()
X=X.reshape(-1,1)

In [18]:
X=np.concatenate(np.concatenate(X,axis=0),axis=0).reshape(-1,300)
y=data['label']
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2)

Machine Learning Algorithms like logistic, SVM etc.,

In [19]:
clf = LogisticRegression()
clf.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Accuracy of the given data**

In [20]:
y_pred=clf.predict(test_x)
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       499
           1       0.99      0.95      0.97       101

    accuracy                           0.99       600
   macro avg       0.99      0.97      0.98       600
weighted avg       0.99      0.99      0.99       600

